<a href="https://colab.research.google.com/github/ShykerBogdan/Prozzoro_project/blob/master/Data_prepr_batches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import time
from datetime import datetime
import numpy as np

In [3]:
filename='/content/gdrive/My Drive/ML_proz/data_source.csv'
df=pd.read_csv(filename)
df.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5,14,15,16,17,18,20,24,38,48,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3507778, 56)

In [0]:
class DataPreproccessing:
    def __init__(self, data_source, churn_interval, start_period=None, end_period=None):
        if start_period:
          data_source=data_source[pd.to_datetime(data_source['date_from_id'])>start_period]
          self.last_day = pd.to_datetime('2018-11-29')
        # для статистики постачальника вибираємо проміжок [start_period,end_period]
        self.divide_data = self.last_day - pd.DateOffset(days=churn_interval)
        print(self.divide_data)
        self.data_source = data_source[pd.to_datetime(data_source['date_from_id']) <= self.divide_data]
        # для labeling вибираємо період [end_period, end_period+churn_interval]
        self.label_data = data_source[pd.to_datetime(data_source['date_from_id']) > self.divide_data]
        # вибираємо унікальних постачальників
        self.unique_id = list(self.data_source['participants'].value_counts().index)
        # створюємо dataframe з колонкою унікальних постачальників і туди будемо додавати features
        self.feature_data = pd.DataFrame(self.unique_id, columns=['unique_id'])

    def activity_date(self):
        f = ['winner', 'tenders.auctionPeriod_startDate', 'contracts.dateSigned', 'date_from_id', 'awards.date']
        self.data_source['new_date'] = self.data_source[f].apply(
            lambda x: x[f[2]] if x[f[0] == 1] else (x[f[3]] if pd.isnull(x[f[1]]) else x[f[1]]), axis=1)
        self.data_source['new_date'] = self.data_source[['new_date']].apply(lambda x: x['new_date'][:10], axis=1)
        self.data_source['new_date'] = self.data_source[['new_date', 'date_from_id']].apply(
            lambda x: x['date_from_id'] if x['new_date'] == '\\N' else x['new_date'], axis=1)

    def cleaning(self):
        self.data_source['winner'] = self.data_source[['contracts.value_amount', 'contracts.status', 'winner']].apply \
            (lambda x: 0 if x['contracts.value_amount'] == '\\N'
                            or x['contracts.status'] == 'cancelled' else x['winner'], axis=1)
        print('winner updated')
        self.activity_date()
        print('activity date')

        def convert_to_float(raw):
            try:
                raw['lots.value_amount'] = float(raw['lots.value_amount'])
            except:
                raw['lots.value_amount'] = np.nan
            return raw

        self.data_source['lots.value_amount'] = self.data_source[['lots.value_amount']].apply(convert_to_float, axis=1)
        print('convert lots')

    def split_into_batches(self, n=20000, filename=None):
        self.cleaning()
        feature_batches = list()

        n_iter = int(np.ceil(self.feature_data.shape[0] / n))
        for i in range(n_iter):
            print('Batch {0}-{1}:'.format(i * n, (i + 1) * n))
            unique_id = self.feature_data.iloc[i * n:(i + 1) * n]['unique_id'].values
            feature_df = pd.DataFrame(unique_id, columns=['unique_id'])
            feature_batches.append(feature_df)
            # part of data_source
            df_ = self.data_source.loc[self.data_source['participants'].isin(unique_id)]
            print('Unique_supp:{0}-{1}'.format(len(unique_id), df_.shape[0]))
            df_ = df_.sort_values(by='new_date')
            self.divide_data = str(self.divide_data)[:10]

            # features:
            # 1. Last activity
            def get_last_activity(raw, data):
                raw['last_activity_date'] = data[data['participants'] == raw['unique_id']].iloc[-1]['new_date']
                try:
                    raw['last_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(raw['last_activity_date'])).days
                except:
                    print('--------------')
                    raw['last_activity_days'] = '?'
                return raw

            feature_batches[i] = feature_batches[i].apply(get_last_activity, args=(df_[['participants','new_date']],),
                                                          axis=1)
            feature_batches[i]['last_activity_days'] = feature_batches[i].apply(
                lambda x: 0 if x['last_activity_days'] < 0 else x['last_activity_days'], axis=1)

            # 2. Average activity
#             def get_average_activity(raw, data):
#                 dates_ = list(data[data['participants'] == raw['unique_id']]['new_date'].values)
#                 try:
#                     raw['average_activity'] = (pd.to_datetime(dates_[-1]) - pd.to_datetime(dates_[0])).days / len(dates_)
#                 except:
#                     raw['average_activity'] = np.nan
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(get_average_activity, args=(df_[['participants','new_date']],),
#                                                           axis=1)

            # 3. Count lots
            freq = df_['participants'].value_counts()

            def count_lots(raw, frequency):
                raw['count_lots'] = frequency[raw['unique_id']]
                return raw

            feature_batches[i] = feature_batches[i].apply(count_lots, args=(freq,), axis=1)

            # 4. Win, lose
            frequency = df_[df_['winner'] == 1]['participants'].value_counts()

            def win_lose(raw, frequency):
                # use try bsc if index not in frequency 'win'=0
                try:
                    raw['win'] = frequency[raw['unique_id']]
                except:
                    raw['win'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(win_lose, args=(
                frequency,), axis=1)
            feature_batches[i]['lose'] = feature_batches[i]['count_lots'] - feature_batches[i]['win']
            # 5. Count win open

#             frequency = df_.loc[(df_['winner'] == 1) & (df_['tenders.procurementMethod'] == 'open')][
#                 'participants'].value_counts()

#             def count_win_open(raw, frequency):
#                 try:
#                     raw['win_open'] = frequency[raw['unique_id']]
#                 except:
#                     raw['win_open'] = 0
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(count_win_open, args=(
#                 frequency,), axis=1)
#             feature_batches[i]['win_not_open'] = feature_batches[i]['win'] - feature_batches[i]['win_open']
            # 6. Count lose open
#             frequency = df_.loc[(df_['winner'] == 0) & (df_['tenders.procurementMethod'] == 'open')][
#                 'participants'].value_counts()

#             def count_lose_open(raw, frequency):
#                 try:
#                     raw['lose_open'] = frequency[raw['unique_id']]
#                 except:
#                     raw['lose_open'] = 0
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(count_lose_open, args=(
#                 frequency,), axis=1)
#             feature_batches[i]['lose_not_open'] = feature_batches[i]['lose'] - feature_batches[i]['lose_open']
            # 7. Average economy
#             data = df_[df_['winner'] == 1][
#                 ['participants', 'awards.value_amount', 'lots.value_amount']]
#             data['economy_value'] = data['lots.value_amount'] - data['awards.value_amount']
#             data['economy_percent'] = data['awards.value_amount'] / data['lots.value_amount']

#             def get_economy(raw, data):
#                 d = data[data['participants'] == raw['unique_id']]
#                 raw['economy_value'] = np.nanmean(d['economy_value'])
#                 raw['economy_percent'] = np.nanmean(d['economy_percent'])
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(get_economy, args=(data,), axis=1)
#             8. Get label
            participants = list(self.label_data['participants'].value_counts().index)

            def get_y(raw, participants):
              if raw['unique_id'] in participants:
                  raw['y'] = 1
              else:
                  raw['y'] = 0
              return raw

            feature_batches[i] = feature_batches[i].apply(get_y, args=(participants,), axis=1)
        if filename:
            print(f'write to csv')
            feature_batches[i].to_csv(filename + '{0}'.format(i) + '.csv', index=False)

        # concat dataframes into one
        print('Concat')
        all_batches_df = pd.concat(feature_batches)
        self.feature_data = all_batches_df.copy()
        if filename:
            print('write to csv_all')
            all_batches_df.to_csv(filename + '_all.csv', index=False)

    def get_label(self):
        participants = list(self.label_data['participants'].value_counts().index)

        def get_y(raw, participants):
            if raw['unique_id'] in participants:
                raw['y'] = 1
            else:
                raw['y'] = 0
            return raw

        self.feature_data = self.feature_data.apply(get_y, args=(participants,), axis=1)


In [0]:
col=['winner','tenders.auctionPeriod_startDate','contracts.dateSigned',
     'awards.date','participants','date_from_id','contracts.value_amount','contracts.status','lots.value_amount']

In [0]:
start_time = time.time()
df1=df[col]
prepr = DataPreproccessing(df1,90,start_period='2016-08-01')
prepr.split_into_batches(n=10000,filename='baseline')
print("--- %s seconds ---" % (time.time() - start_time))
print(prepr.feature_data.head())
# 158 sec - 30000

2018-08-31 00:00:00
winner updated
activity date
